<img src="UNAV.png" width=100 height=100 align="right">

# Python Ecosystem -  Trabajando con ficheros de datos - Numpy & Pandas

<img src="pwc_logo.png" width=120 height=120 align="right">

<img src="profile_manoelgadi.png" width=100 height=100 align="right">

Author: Prof. Manoel Gadi

Contact: manoelgadi@gmail.com

Teaching Web: http://mfalonso.pythonanywhere.com

Last revision: 18/June/2019

Date: 07/February/2018
    
Author: Prof. Manoel Gadi - http://mfalonso.pythonanywhere.com/ - https://www.linkedin.com/in/manoel-gadi-97821213/


## Parte 2: Trabajando con Data Frames para calcular la Beta de mercado


<img src="BetaCalculation.png" align="center">




* Referencia 1: http://gouthamanbalaraman.com/blog/calculating-stock-beta.html
* Referencia 2: http://www.investopedia.com/ask/answers/070615/what-formula-calculating-beta.asp

In [1]:
!pip install pandas_datareader

In [2]:
!pip install fix_yahoo_finance

  Created wheel for fix-yahoo-finance: filename=fix_yahoo_finance-0.1.35-py3-none-any.whl size=12443 sha256=38a333bc09572c4d20f5ed2a57e91cb158d32df60bb14725235d9eb536ff1b31
  Stored in directory: c:\users\incompany\appdata\local\pip\cache\wheels\ec\8b\42\b4b6c3019ecbb6e352f974109b8c971bdf7f9dba0278997aca
  Created wheel for multitasking: filename=multitasking-0.0.10-py3-none-any.whl size=8500 sha256=9234c86f7099beda7aa8e85c41fa371af8dc001efd12687bcfa249fa5a4656de
  Stored in directory: c:\users\incompany\appdata\local\pip\cache\wheels\f2\b5\2c\59ba95dcf854e542944c75fe3da584e4e3833b319735a0546c
Successfully built fix-yahoo-finance multitasking


    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\incompany\Anaconda3\python.exe' -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\incompany\\AppData\\Local\\Temp\\pip-install-5g0g16kz\\fix-yahoo-finance_662357f2d5ed474f8b76262f6dacc6fd\\setup.py'"'"'; __file__='"'"'C:\\Users\\incompany\\AppData\\Local\\Temp\\pip-install-5g0g16kz\\fix-yahoo-finance_662357f2d5ed474f8b76262f6dacc6fd\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\incompany\AppData\Local\Temp\pip-pip-egg-info-d9yl0gqj'
         cwd: C:\Users\incompany\AppData\Local\Temp\pip-install-5g0g16kz\fix-yahoo-finance_662357f2d5ed474f8b76262f6dacc6fd\
    Complete output (7 lines):
    Traceback (most recent call last):
      Fil

In [3]:
import pandas_datareader.data as web
import fix_yahoo_finance as yf
import datetime
import numpy as np
import pandas as pd

### Descargando datos historicos

Capturando datos de serie historica de 5 años para la acción de Telefónica (TEF.MC) y para el índice IBEX 35

In [4]:
fecha_final =   datetime.datetime(2019,12, 31, 0, 0, 0, 0)
fecha_inicial = fecha_final - datetime.timedelta(days=5*365)

In [5]:
df = web.get_data_yahoo("SPY", start=fecha_inicial, end=fecha_final)

In [6]:
fecha_final, fecha_inicial

(datetime.datetime(2019, 12, 31, 0, 0), datetime.datetime(2015, 1, 1, 0, 0))

In [7]:
ticker_symbol = 'TEF.MC'
ref_index = '^IBEX'

ticker_symbol, ref_index

('TEF.MC', '^IBEX')

Leyendo los datos de la API de https://finance.yahoo.com/?guccounter=1

In [8]:
df_stock = web.get_data_yahoo(ticker_symbol,fecha_inicial,fecha_final)

In [9]:
df_stock.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-01-02,11.903631,11.641633,11.864085,11.750387,24004852.0,7.400207
2015-01-05,11.765217,11.295597,11.641633,11.354918,31002826.0,7.151144
2015-01-06,11.419181,11.206616,11.414238,11.241220,29842211.0,7.079541
2015-01-07,11.409295,11.181900,11.295597,11.246163,22330027.0,7.082653
2015-01-08,11.552652,11.295597,11.399408,11.522992,20183081.0,7.256995


Haciendo lo mismo para el indice de referencia

In [10]:
df_index = web.get_data_yahoo(ref_index,fecha_inicial,fecha_final)

In [11]:
df_index.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-01-02,10440.299805,10232.299805,10386.700195,10350.799805,220239700.0,10350.789062
2015-01-05,10390.799805,9977.799805,10267.200195,9993.299805,299610800.0,9993.290039
2015-01-06,10060.799805,9871.099609,10040.700195,9871.099609,282855400.0,9871.089844
2015-01-07,10051.200195,9836.400391,9937.299805,9891.400391,290122400.0,9891.390625
2015-01-08,10143.000000,9970.299805,10053.200195,10115.000000,320452300.0,10114.990234


### Para calcular la Beta necesitamos crear una serie temporal de puntos de datos mensuales

In [12]:
df_stock = df_stock.resample('M').last()

In [13]:
df_stock.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-01-31,13.268001,13.050493,13.243284,13.154304,28201356.0,8.284369
2015-02-28,13.762338,13.544830,13.693131,13.737621,27297408.0,8.651734
2015-03-31,13.340000,13.125000,13.330000,13.260000,26050666.0,8.350936
2015-04-30,13.725000,13.475000,13.590000,13.640000,114188383.0,8.590255
2015-05-31,13.070000,12.835000,13.025000,12.880000,24680620.0,8.352767


Haciendo lo mismo para el indice de referencia

In [14]:
df_index = df_index.resample('M').last()

In [15]:
df_index.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-01-31,10581.599609,10336.099609,10562.599609,10403.299805,418610700.0,10403.289062
2015-02-28,11184.500000,11070.799805,11142.900391,11178.299805,412536300.0,11178.288086
2015-03-31,11596.099609,11444.599609,11510.799805,11521.099609,336270600.0,11521.087891
2015-04-30,11469.200195,11258.000000,11351.099609,11385.000000,386580900.0,11384.988281
2015-05-31,11402.700195,11192.099609,11375.599609,11217.599609,327769600.0,11217.587891


### Para calcular la Beta utilizamos el percentual de variación mes a mes, no el valor de la acción directamente

In [16]:
df_stock['returns'] = df_stock['Adj Close']/ df_stock['Adj Close'].shift(1) -1

In [17]:
df_stock['returns'].head()

Date
2015-01-31         NaN
2015-02-28    0.044344
2015-03-31   -0.034767
2015-04-30    0.028658
2015-05-31   -0.027646
Freq: M, Name: returns, dtype: float64

In [18]:
df_stock = df_stock.dropna()

In [19]:
df_stock['returns'].head()

Date
2015-02-28    0.044344
2015-03-31   -0.034767
2015-04-30    0.028658
2015-05-31   -0.027646
2015-06-30   -0.010093
Freq: M, Name: returns, dtype: float64

In [20]:
df_stock.head()

,High,Low,Open,Close,Volume,Adj Close,returns
Date,,,,,,,
2015-02-28,13.762338,13.54483,13.693131,13.737621,27297408.0,8.651734,0.044344
2015-03-31,13.340000,13.12500,13.330000,13.260000,26050666.0,8.350936,-0.034767
2015-04-30,13.725000,13.47500,13.590000,13.640000,114188383.0,8.590255,0.028658
2015-05-31,13.070000,12.83500,13.025000,12.880000,24680620.0,8.352767,-0.027646
2015-06-30,13.010000,12.73000,12.870000,12.750000,32148269.0,8.268459,-0.010093


Haciendo lo mismo para el indice de referencia

In [21]:
df_index['returns'] = df_index['Adj Close']/ df_index['Adj Close'].shift(1) -1
df_index = df_index.dropna()


In [22]:
df_index.head()

,High,Low,Open,Close,Volume,Adj Close,returns
Date,,,,,,,
2015-02-28,11184.500000,11070.799805,11142.900391,11178.299805,412536300.0,11178.288086,0.074496
2015-03-31,11596.099609,11444.599609,11510.799805,11521.099609,336270600.0,11521.087891,0.030667
2015-04-30,11469.200195,11258.000000,11351.099609,11385.000000,386580900.0,11384.988281,-0.011813
2015-05-31,11402.700195,11192.099609,11375.599609,11217.599609,327769600.0,11217.587891,-0.014704
2015-06-30,10954.500000,10723.200195,10811.599609,10769.500000,369234200.0,10769.489258,-0.039946


### Haciendo merge de los 2 dataframes por fecha (el index del dataframe es la fecha) y cogiendo solo las columnas stock_returns y index_returns

In [23]:
df = pd.DataFrame({'stock_returns' : df_stock['returns'],
                        'index_returns' : df_index['returns']},
                        index=df_stock.index)

In [24]:
df.head()

,stock_returns,index_returns
Date,,
2015-02-28,0.044344,0.074496
2015-03-31,-0.034767,0.030667
2015-04-30,0.028658,-0.011813
2015-05-31,-0.027646,-0.014704
2015-06-30,-0.010093,-0.039946


Por segurida borramos las filas que no tienen los 2 valores por si hubiera.

In [25]:
df = df.dropna()

In [26]:
df.head()

,stock_returns,index_returns
Date,,
2015-02-28,0.044344,0.074496
2015-03-31,-0.034767,0.030667
2015-04-30,0.028658,-0.011813
2015-05-31,-0.027646,-0.014704
2015-06-30,-0.010093,-0.039946


#### Implementando la función para la covarianza

Referencia: http://ci.columbia.edu/ci/premba_test/c0331/s7/s7_5.html

In [27]:
def covariance(a, b):
    if len(a) != len(b):
        return
    a_mean = np.mean(a)
    b_mean = np.mean(b)
    sum = 0
    for i in range(0, len(a)):
        sum += ((a[i] - a_mean) * (b[i] - b_mean))
    return sum/(len(a)-1)

Miremos otra vez a la formula de la Beta:
    
<img src="BetaCalculation.png" width=400 height=400 align="center">

In [28]:
numerator = covariance(df['stock_returns'],df['index_returns'])
print("COVARIANCE(stock, benchmark) = COVARIANCE("
      +ticker_symbol+", "+ref_index +") = " +str(numerator))

COVARIANCE(stock, benchmark) = COVARIANCE(TEF.MC, ^IBEX) = 0.002077376478545967


In [29]:
denominator = covariance(df['index_returns'],df['index_returns'])
print("VARIANCE(benchmark) = COVARIANCE(benchmark, benchmark) = COVARIANCE("
      +ref_index+", "+ref_index +") = " +str(denominator))


VARIANCE(benchmark) = COVARIANCE(benchmark, benchmark) = COVARIANCE(^IBEX, ^IBEX) = 0.001965250097421376


In [30]:
# BETA = Covariance (stock,index) / Variance (Index) = Covariance (stock,index) / Covariance (stock,stock)
print("BETA = COVARIANCE(stock, benchmark) / VARIANCE(benchmark) = " 
      + str(numerator) + " / " + str(denominator) + " = " 
      +str(covariance(df['stock_returns'],df['index_returns'])
           /covariance(df['index_returns'],df['index_returns'])))

BETA = COVARIANCE(stock, benchmark) / VARIANCE(benchmark) = 0.002077376478545967 / 0.001965250097421376 = 1.0570545098925135


<img src="curso.jpeg" width=300 height=300 align="center">

<img src="UNAV.png" width=100 height=100 align="right">

<img src="pwc_logo.png" width=120 height=120 align="right">

In [ ]:
# Librerias
# !pip install pandas_datareader
# !pip install fix_yahoo_finance

In [35]:
import pandas_datareader.data as web
import fix_yahoo_finance as yf
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

fecha_final =   datetime.today() 
fecha_inicial = fecha_final - timedelta(days=5*365) 

ticker_symbol = 'TEF.MC'
ref_index = '^IBEX'
df_stock = web.get_data_yahoo(ticker_symbol,fecha_inicial,fecha_final)
df_index = web.get_data_yahoo(ref_index,fecha_inicial,fecha_final)
df_stock = df_stock.resample('M').last() #wip
df_index = df_index.resample('M').last() #wip
df_stock['returns'] = df_stock['Adj Close']/ df_stock['Adj Close'].shift(1) -1
df_stock = df_stock.dropna()
df_index['returns'] = df_index['Adj Close']/ df_index['Adj Close'].shift(1) -1
df_index = df_index.dropna()
df = pd.DataFrame({'stock_returns' : df_stock['returns'],
                        'index_returns' : df_index['returns']},
                        index=df_stock.index)
df = df.dropna()

def covariance(a, b):
    if len(a) != len(b):
        return
    a_mean = np.mean(a)
    b_mean = np.mean(b)
    sum = 0
    for i in range(0, len(a)):
        sum += ((a[i] - a_mean) * (b[i] - b_mean))
    return sum/(len(a)-1)

numerator = covariance(df['stock_returns'],df['index_returns'])
denominator = covariance(df['index_returns'],df['index_returns'])

print("BETA = COVARIANCE(stock, benchmark) / VARIANCE(benchmark) = " 
      + str(numerator) + " / " + str(denominator) + " = " 
      +str(covariance(df['stock_returns'],df['index_returns'])
           /covariance(df['index_returns'],df['index_returns'])))

BETA = COVARIANCE(stock, benchmark) / VARIANCE(benchmark) = 0.0033783371675713617 / 0.003223570949539044 = 1.0480107993448844


In [56]:
def calcula_beta(ticker_symbol = 'TEF.MC', ref_index = '^IBEX'):
    """
    Esta función devuelve la beta mensual de 5 años del mercado entre una acción y un índice de referencia. 
    """
    import pandas_datareader.data as web
    import fix_yahoo_finance as yf
    from datetime import datetime, timedelta
    import numpy as np
    import pandas as pd


    fecha_final =   datetime.today() 
    fecha_inicial = fecha_final - timedelta(days=5*365) 
    try:
        df_stock = web.get_data_yahoo(ticker_symbol,fecha_inicial,fecha_final)
        df_index = web.get_data_yahoo(ref_index,fecha_inicial,fecha_final)
    except:
        return("Problema de conexión con Yahoo. Hablar con Tecnología.")
    
    df_stock = df_stock.resample('M').last() #wip
    df_index = df_index.resample('M').last() #wip
    df_stock['returns'] = df_stock['Adj Close']/ df_stock['Adj Close'].shift(1) -1
    df_stock = df_stock.dropna()
    df_index['returns'] = df_index['Adj Close']/ df_index['Adj Close'].shift(1) -1
    df_index = df_index.dropna()
    df = pd.DataFrame({'stock_returns' : df_stock['returns'],
                            'index_returns' : df_index['returns']},
                            index=df_stock.index)
    df = df.dropna()

    def covariance(a, b):
        if len(a) != len(b):
            return
        a_mean = np.mean(a)
        b_mean = np.mean(b)
        sum = 0
        for i in range(0, len(a)):
            sum += ((a[i] - a_mean) * (b[i] - b_mean))
        return sum/(len(a)-1)

    numerator = covariance(df['stock_returns'],df['index_returns'])
    denominator = covariance(df['index_returns'],df['index_returns'])
    try:
        return(numerator/denominator)
    except:
        return("El índice tiene covarianza 0. División por 0.")

In [57]:
help(calcula_beta)

Help on function calcula_beta in module __main__:

calcula_beta(ticker_symbol='TEF.MC', ref_index='^IBEX')
    Esta función devuelve la beta mensual de 5 años del mercado entre una acción y un índice de referencia.



In [53]:
calcula_beta("REP.MC")

1.3865883310320064

In [54]:
calcula_beta("GOOG", "SPY")

1.0649951614980124

In [55]:
calcula_beta("ASD")

'ERROR.'